In [0]:
import torch
from torch.autograd import Function
class PlusOne(Function):
  @staticmethod
  def forward(ctx, i):
    return i +1

  @staticmethod
  def backward(ctx, grad_output):
    return grad_output

In [0]:
from torch.autograd import gradcheck
input = torch.randn(20,20,dtype=torch.double,requires_grad=True)
print(gradcheck(PlusOne.apply, input, eps=1e-6, atol=1e-4))

True


In [0]:
class LayerPlusOne(torch.nn.Module):
    def __init__(self):
        super(LayerPlusOne, self).__init__()

    def forward(self, input):
        return PlusOne.apply(input)

In [0]:
a = torch.randn(2, 2, 2, dtype=torch.double)
print(a)
plus_one = LayerPlusOne()
print(plus_one(a))

tensor([[[ 1.2557, -0.0767],
         [ 0.7491, -0.8358]],

        [[-0.6017, -0.6456],
         [-1.3795,  0.1545]]], dtype=torch.float64)
tensor([[[ 2.2557,  0.9233],
         [ 1.7491,  0.1642]],

        [[ 0.3983,  0.3544],
         [-0.3795,  1.1545]]], dtype=torch.float64)


In [0]:
%%bash
cd cpp
ls -l
python setup.py install

total 12
drwxr-xr-x 1 root root 4096 Apr  3 16:24 sample_data
-rw-r--r-- 1 root root  265 Apr 14 07:31 setup.py
-rw-r--r-- 1 root root  370 Apr 14 07:31 tensor_addition.cpp
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
running install
running bdist_egg
running egg_info
creating tensor_addition.egg-info
writing tensor_addition.egg-info/PKG-INFO
writing dependency_links to tensor_addition.egg-info/dependency_links.txt
writing top-level names to tensor_addition.egg-info/top_level.txt
writing manifest file 'tensor_addition.egg-info/SOURCES.txt'
reading manifest file 'tensor_addition.egg-info/SOURCES.txt'
writing manifest file 'tensor_addition.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_ext
building 'tensor_addition' extension
creating build
creating build/temp.linux-x86_64-3.6
x86_64-linux-gnu-gcc -pthread -DNDEBUG -g -fwrapv -O2 -Wall -g -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -

zip_safe flag not set; analyzing archive contents...
__pycache__.tensor_addition.cpython-36: module references __file__


In [0]:
import sys
egg_path='/usr/local/lib/python3.6/dist-packages/tensor_addition-0.0.0-py3.6-linux-x86_64.egg'
sys.path.append(egg_path)
import tensor_addition

help(tensor_addition.add_forward)

Help on built-in function add_forward in module tensor_addition:

add_forward(...) method of builtins.PyCapsule instance
    add_forward(arg0: at::Tensor, arg1: at::Tensor) -> at::Tensor
    
    Add forward



In [0]:
from torch.autograd import Function
class AdditionFunction(Function):
    @staticmethod
    def forward(ctx, input):
        outputs = tensor_addition.add_forward(input,torch.ones(input.size()))
        return outputs
    @staticmethod
    def backward(ctx, grad_output):
        return tensor_addition.add_backward(grad_output)
        

In [0]:
input = torch.randn(20,20,dtype=torch.double,requires_grad=True)
print(gradcheck(AdditionFunction.apply, input, eps=1e-6, atol=1e-4))

True


In [0]:
class AdditionLayer(torch.nn.Module):
    def __init__(self):
        super(AdditionLayer, self).__init__()
    def forward(self, input):
        return AdditionFunction.apply(input) 
a = torch.randn(2, 2, 2, dtype=torch.double)
print(a)
adl = AdditionLayer()
plus_one = adl.forward(a)
print(plus_one)    

tensor([[[ 1.5470,  0.7305],
         [ 0.2786, -0.2576]],

        [[ 1.2554, -0.3133],
         [-0.1350,  0.1036]]], dtype=torch.float64)
tensor([[[2.5470, 1.7305],
         [1.2786, 0.7424]],

        [[2.2554, 0.6867],
         [0.8650, 1.1036]]], dtype=torch.float64)
